In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.models import TextClassifier

class FlairTrader:
    def __init__(self, company_name, ticker):
        self.company_name = company_name
        self.ticker = ticker
        self.tagger = SequenceTagger.load('ner')
        self.classifier = TextClassifier.load('en-sentiment')
        
        
    def _references_company(self, text):
        sentence = Sentence(text)
        self.tagger.predict(sentence)

        for token in sentence.get_labels('ner'):
            if (token.value == "ORG" and
                token.span.text == self.company_name):
                return True

        return False
    
    def buy_or_sell(self, text):
        if self._references_company(text):
            sentence = Sentence(text)
            self.classifier.predict(sentence)

            if sentence.labels[0].value == "POSITIVE":
                return "buy"
            elif sentence.labels[0].value == "NEGATIVE":
                return "sell"
        else:
            return "hold"

In [ ]:
import pandas as pd

url = ("https://github.com/PacktPublishing/Natural-Language-"
       "Processing-with-Flair/blob/nlp-with-flair/data/milli"
       "on_articles_cc0_license.csv?raw=true")

df = pd.read_csv(url, parse_dates=['publish_date'])
df = df[df['headline_text'].str.contains('tesla')]
df = df.drop_duplicates(subset=['publish_date'])
df["publish_date"] = df["publish_date"].astype(str)
df = df.set_index("publish_date")

headlines = df["headline_text"].to_dict()

In [ ]:
import yfinance as yf
import backtrader as bt

yf_dataset = yf.download('TSLA',
                         '2016-01-01',
                         '2017-01-01')

stock_data = bt.feeds.PandasData(dataname=yf_dataset)

In [ ]:
from datetime import datetime
import backtrader as bt
import random

class FlairTradeStrategy(bt.Strategy):
    params = (('flair_trader', None), ('headlines', None))
    
    def next(self):
        headline = self.p.headlines.get(
            str(self.datetime.date()))
        if headline:
            action = self.p.flair_trader.buy_or_sell(headline)

            if action == "buy":
                self.buy()
            if action == "sell":
                self.sell()        

In [ ]:
trader = bt.Cerebro()
trader.broker.setcash(100)
trader.adddata(stock_data)

flair_trader = FlairTrader("tesla", "TSLA")
trader.addstrategy(FlairTradeStrategy,
                   flair_trader=flair_trader,
                   headlines=headlines)
trader.run()
trader.plot()[0][0].savefig('result.png', dpi=300)